In [3]:
from belief_base import BeliefBase
from contraction import BeliefContraction
from expansion import BeliefExpansion
from entailment import Resolution

def test_entailment(belief_base, formula):
    print(f"Checking entailment: Does Belief Base entail '{formula}'?")
    result = Resolution.entails(belief_base.list_beliefs(), formula)
    print(f"Result: {'YES' if result else 'NO'}\n")
    return result

def test_agm_postulates(belief_base, contraction_agent, formula):
    print(f"\n--- Testing AGM Postulates for contraction by '{formula}' ---")
    original_beliefs = belief_base.list_beliefs()

    # 1. Success
    contraction_agent.contract(formula)
    success = not Resolution.entails(belief_base.list_beliefs(), formula)
    print(f"Success Postulate: {'PASSED' if success else 'FAILED'}")

    # 2. Inclusion
    inclusion = set(belief_base.list_beliefs()).issubset(set(original_beliefs))
    print(f"Inclusion Postulate: {'PASSED' if inclusion else 'FAILED'}")

    # 3. Vacuity
    temp_bb = BeliefBase()
    for b in original_beliefs:
        temp_bb.add_belief(b)
    temp_contractor = BeliefContraction(temp_bb)
    if not Resolution.entails(temp_bb.list_beliefs(), "Z"):
        before = set(temp_bb.list_beliefs())
        temp_contractor.contract("Z")
        after = set(temp_bb.list_beliefs())
        vacuity = before == after
        print(f"Vacuity Postulate: {'PASSED' if vacuity else 'FAILED'}")
    else:
        print("Vacuity Postulate: SKIPPED (Z was entailed)")

    # 4. Consistency
    is_consistent = Resolution.is_consistent(belief_base.list_beliefs())
    print(f"Consistency Postulate: {'PASSED' if is_consistent else 'FAILED'}")

    # 5. Extensionality
    temp_bb1 = BeliefBase()
    temp_bb2 = BeliefBase()

    for b in original_beliefs:
        temp_bb1.add_belief(b)
        temp_bb2.add_belief(b)

    contractor1 = BeliefContraction(temp_bb1)
    contractor2 = BeliefContraction(temp_bb2)

    # Logical equivalents: (A ∨ B) ≡ (B ∨ A)
    equivalent_formula = formula  # You can adjust this if you want

    contractor1.contract(formula)
    contractor2.contract(equivalent_formula)

    base1 = set(temp_bb1.list_beliefs())
    base2 = set(temp_bb2.list_beliefs())

    extensionality = base1 == base2
    print(f"Extensionality Postulate: {'PASSED' if extensionality else 'FAILED'}")
    

def full_belief_management_test():
    print("\n=== Creating the Belief Base ===")
    bb = BeliefBase()

    # Adding initial 44 beliefs
    initial_beliefs = [
        "(A ∧ B)", "(¬A ∨ C)", "(¬B ∧ D)", "(E ∨ F)", "(¬E ∨ G)",
        "(H ∨ I)", "(¬H ∨ J)", "(K ∨ L)", "(¬K ∨ M)", "(N ∨ O)",
        "(P ∨ Q)", "(¬P ∨ R)", "(S ∨ T)", "(U ∨ V)", "(W ∨ X)",
        "(Y ∨ Z)", "(¬Y ∨ A)", "(¬X ∨ B)", "(¬W ∨ C)", "(¬U ∨ D)",
        "(¬A ∨ (B ∧ C))", "((D ∧ E) ∨ (F ∧ G))", "(H ∨ (I ∧ J))",
        "((K ∨ L) ∧ (M ∨ ¬N))", "(O ∧ (P ∨ Q))", "(¬R ∨ (S ∧ T))",
        "((U ∨ V) ∧ (¬W ∨ X))", "(Y ∨ (Z ∧ A))", "((B ∧ C) ∨ (D ∧ ¬E))",
        "(F ∨ (G ∧ H ∨ I))", "(J ∨ (¬K ∧ L))", "((M ∧ N) ∨ (O ∨ ¬P))",
        "(Q ∨ (R ∧ (S ∨ T)))", "((U ∨ V) ∧ (W ∨ ¬X))", "(Y ∧ (Z ∨ (A ∧ B)))",
        "(¬(C ∧ D) ∨ (E ∨ F))", "((G ∧ ¬H) ∨ (I ∧ J))", "(K ∨ (L ∧ (M ∨ ¬N)))",
        "((O ∨ P) ∧ (¬Q ∨ R))", "(S ∧ (T ∨ U))", "(V ∨ (W ∧ ¬X))",
        "((Y ∧ Z) ∨ (A ∧ B))", "(¬(C ∧ D) ∨ (E ∧ F))", "(G ∨ (H ∧ (I ∨ J)))"
    ]
    for belief in initial_beliefs:
        bb.add_belief(belief)

    print(bb)
    print(f"\nTotal beliefs after initial addition: {len(bb)}")

    # --- Force some entailments ---
    print("\n--- Expanding Belief Base with extra facts to force entailments ---")
    expander = BeliefExpansion(bb)
    expander.expand("(¬B)")  # To force A from (A ∨ B)
    expander.expand("(¬F)")  # To force E from (E ∨ F)
    expander.expand("(¬Z)")  # To force Y from (Y ∨ Z)
    expander.expand("(¬D)")  # To force C from (¬W ∨ C) and related
    expander.expand("(¬G)")  # To help E or F cases

    print("\nAfter extra expansion:")
    print(bb)
    print(f"\nTotal beliefs after expansion: {len(bb)}")

    # --- Testing Entailments ---
    print("\n--- Testing Entailments ---")
    test_entailment(bb, "A")  # Should be YES now
    test_entailment(bb, "B")  # Should be NO (negated B)
    test_entailment(bb, "C")  # Should be YES
    test_entailment(bb, "D")  # Should be NO (negated D)
    test_entailment(bb, "E")  # Should be YES

    # --- Contraction ---
    print("\n--- Contracting Belief Base ---")
    priorities = {
        "(A ∧ B)": 5,
        "(¬A ∨ C)": 4,
        "(¬B ∧ D)": 4,
        "(E ∨ F)": 2,
        "(¬E ∨ G)": 1,
        "(¬D)": 3,
        "(M ∨ N)": 2,
        "(¬M ∨ O)": 2,
        "(¬N ∨ P)": 2
    }
    contractor = BeliefContraction(bb, priorities)
    contractor.contract("D")
    print("\nAfter Contraction:")
    print(bb)
    print(f"\nTotal beliefs after contraction: {len(bb)}")

    # --- Test AGM Postulates ---
    test_agm_postulates(bb, contractor, "D")


def main():
    print("\n===== BELIEF REVISION AGENT =====\n")
    full_belief_management_test()
    print("\n===== TERMINATED =====\n")

if __name__ == "__main__":
    main()


===== BELIEF REVISION AGENT =====


=== Creating the Belief Base ===
Belief Base:
- ((B ∧ C) ∨ (D ∧ ¬E))
- ((D ∧ E) ∨ (F ∧ G))
- ((G ∧ ¬H) ∨ (I ∧ J))
- ((K ∨ L) ∧ (M ∨ ¬N))
- ((M ∧ N) ∨ (O ∨ ¬P))
- ((O ∨ P) ∧ (¬Q ∨ R))
- ((U ∨ V) ∧ (W ∨ ¬X))
- ((U ∨ V) ∧ (¬W ∨ X))
- ((Y ∧ Z) ∨ (A ∧ B))
- (A ∧ B)
- (E ∨ F)
- (F ∨ (G ∧ H ∨ I))
- (G ∨ (H ∧ (I ∨ J)))
- (H ∨ (I ∧ J))
- (H ∨ I)
- (J ∨ (¬K ∧ L))
- (K ∨ (L ∧ (M ∨ ¬N)))
- (K ∨ L)
- (N ∨ O)
- (O ∧ (P ∨ Q))
- (P ∨ Q)
- (Q ∨ (R ∧ (S ∨ T)))
- (S ∧ (T ∨ U))
- (S ∨ T)
- (U ∨ V)
- (V ∨ (W ∧ ¬X))
- (W ∨ X)
- (Y ∧ (Z ∨ (A ∧ B)))
- (Y ∨ (Z ∧ A))
- (Y ∨ Z)
- (¬(C ∧ D) ∨ (E ∧ F))
- (¬(C ∧ D) ∨ (E ∨ F))
- (¬A ∨ (B ∧ C))
- (¬A ∨ C)
- (¬B ∧ D)
- (¬E ∨ G)
- (¬H ∨ J)
- (¬K ∨ M)
- (¬P ∨ R)
- (¬R ∨ (S ∧ T))
- (¬U ∨ D)
- (¬W ∨ C)
- (¬X ∨ B)
- (¬Y ∨ A)

Total beliefs after initial addition: 44

--- Expanding Belief Base with extra facts to force entailments ---
Expanded belief base with '(¬B)'.
Expanded belief base with '(¬F)'.
Expanded belief base with '(¬Z)'.
Ex

In [10]:
!jupyter nbconvert --to script expansion.ipynb

[NbConvertApp] Converting notebook expansion.ipynb to script
[NbConvertApp] Writing 563 bytes to expansion.py
